## Crawling 1
- 유튜브를 크롤링하여 데이터 획득
- 크롤링을 해야 하는 이유 : 유튜브 자체가 ajax를 이용한 방식이므로 사람의 손길이 필요한 사이트  
  
- 일반적인 html 자체에는 자체 정의 태그들을 혼용 -> Angualr js를 부분사용했다고 판단된다.
- 일반적인 css selector의 규칙 중에 id 규칙(문서 내에 고유하다)을 무시하므로 사용 시 주의가 필요
- 데이터 로드는 ajax를 통해서 사용자가 스크롤링을 해서 리스트의 끝에 도달할 때 더보기로 처리되므로, 전체 데이터를 수집하기 위해서는 js를 가동시켜야 한다
- 라이브러리는 selenium을 사용 : 웹 브라우저를 컨트롤하여 웹 UI를 Automation 하는 도구 중의 하나  
> https://www.seleniumhq.org/
-> Download  
- 웹 드라이버(여기서는 Chrome을 사용)
> https://sites.google.com/a/chromium.org/chromedriver/downloads  
> 최신 버전이 적용 안 되거나 버그가 있는 기능이 있으므로, 안 되면 버전을 내려가면서 사용한다  
- API-DOCs
> https://seleniumhq.github.io/selenium/docs/api/py/index.html  
> https://selenium-python.readthedocs.io/  
- install  
> conda install selenium  
> or  
> pip install selenium  

In [46]:
from selenium import webdriver as wd

In [47]:
import urllib

In [48]:
# 업로드순 정렬, 오늘(현재 기준 24시간 이내 게시물)
target_url = 'https://www.youtube.com/results?search_query={}&sp=EgIIAg%253D%253D'
target_url

'https://www.youtube.com/results?search_query={}&sp=EgIIAg%253D%253D'

In [49]:
keyword = '갤럭시노트 10'

In [50]:
target_url = target_url.format(urllib.parse.quote(keyword))
target_url

'https://www.youtube.com/results?search_query=%EA%B0%A4%EB%9F%AD%EC%8B%9C%EB%85%B8%ED%8A%B8%2010&sp=EgIIAg%253D%253D'

In [51]:
# 웹 드라이버
driver = wd.Chrome('../tool/chromedriver.exe')

In [52]:
# 사이트 접속
driver.get(target_url)

In [53]:
# 스크롤을 쳐서 데이터를 다 로드시킨다
# 데이터량이 엄청 많지 않을 때 가능
# ajax를 사용할 경우, '서버가 반응해서 응답이 오고 화면을 구성할 때까지' 시간이 걸린다
# -> 스크롤마다 대기시간이 필요
# https://selenium-python.readthedocs.io/waits.html
import time
for i in range(3) :
    driver.execute_script('window.scrollBy(0, 1000)')
    # 다음 작업을 코드로 강제 대기시킨 것
    # 데이터 추출 시에는 selenium에서 지원하는 waits 기법을 사용하면 좋다
    time.sleep(2)

In [54]:
# 영상 데이터 한 줄 : ytd-video-renderer
# 영상 메타 정보 : #video-title

In [55]:
videos = driver.find_elements_by_tag_name('ytd-video-renderer')
len(videos), type(videos)  # 튜플로 리턴

(40, list)

In [56]:
# driver.find_element_by_( )
# 속성의 값은 get_attribute()로 획득

for video in videos[:2] :      # (데이터가 너무 많아서 슬라이싱)
    # 영상 타이틀
    print(video.find_element_by_id('video-title').get_attribute('title'))
    # 썸네일
    print(video.find_element_by_tag_name('img').get_attribute('src'))
    # 링크
    print(video.find_element_by_id('video-title').get_attribute('href'))
    # 작성자
    print(video.find_element_by_css_selector('#byline>a').text)
    # 조회수, 작성시간 : metadata-line>span
    for span in video.find_elements_by_css_selector('#metadata-line>span') :
        print(span.text)

갤럭시 노트10 VS 노트 10+(플러스) 무엇을 사야할까?
https://i.ytimg.com/vi/roVed-6njCE/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCSrKNWv4WHkHP7gWUyb6vMRN1o5Q
https://www.youtube.com/watch?v=roVed-6njCE
기똥찬 똥찬 Amazing ddongchan
조회수 510회
2시간 전
갤럭시 노트 10 플러스 개봉기! 신기능 줌인 마이크 & 에어 모션 확인해봐요!
https://i.ytimg.com/vi/nZrTONlguW8/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCTQbYo0FB8drz84rvOgYn44ZUrZQ
https://www.youtube.com/watch?v=nZrTONlguW8
뻘짓연구소
조회수 8.8천회
18시간 전


In [57]:
'''
[
    {'title' : 'xx', 'thum' : 'xx', 'link' : 'xx', 'author' : 'xx', 'views' : 'xx', 'time' : '' },
    {},
    {},
    ,,,
]
'''

"\n[\n    {'title' : 'xx', 'thum' : 'xx', 'link' : 'xx', 'author' : 'xx', 'views' : 'xx', 'time' : '' },\n    {},\n    {},\n    ,,,\n]\n"

In [58]:
results = []
for video in videos :
    dic = {
        'title' : video.find_element_by_id('video-title').get_attribute('title'),
        'thum' : video.find_element_by_tag_name('img').get_attribute('src'),
        'link' : video.find_element_by_id('video-title').get_attribute('href'),
        'author' : video.find_element_by_css_selector('#byline>a').text,
        'views' : video.find_elements_by_css_selector('#metadata-line>span')[0].text,
        'time' : video.find_elements_by_css_selector('#metadata-line>span')[1].text
    }
    results.append(dic)
results

[{'title': '갤럭시 노트10 VS 노트 10+(플러스) 무엇을 사야할까?',
  'thum': 'https://i.ytimg.com/vi/roVed-6njCE/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCSrKNWv4WHkHP7gWUyb6vMRN1o5Q',
  'link': 'https://www.youtube.com/watch?v=roVed-6njCE',
  'author': '기똥찬 똥찬 Amazing ddongchan',
  'views': '조회수 510회',
  'time': '2시간 전'},
 {'title': '갤럭시 노트 10 플러스 개봉기! 신기능 줌인 마이크 & 에어 모션 확인해봐요!',
  'thum': 'https://i.ytimg.com/vi/nZrTONlguW8/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCTQbYo0FB8drz84rvOgYn44ZUrZQ',
  'link': 'https://www.youtube.com/watch?v=nZrTONlguW8',
  'author': '뻘짓연구소',
  'views': '조회수 8.8천회',
  'time': '18시간 전'},
 {'title': '(광고NO) 갤럭시노트10 사전예약 할까? 아니면 저렴한 노트9을 구매할까? 구매전 필수 체크리스트',
  'thum': 'https://i.ytimg.com/vi/npSRat7apZM/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCKnBCmjvYFfSTw6yb-9d7qxNZPKg',
  'link': 'https://www.youtube.com/watch?v=npSRat7apZM',
  'author': '딴트공말방구 실험실',
  'views': '조회수 4.6만회',
  'tim

In [61]:
for result in results[:1] :
    print(result['link'])
    # 해당 콘텐츠에 진입
    driver.get(result['link'])
    # 잠시 대기 -> 반드시 신규페이지에서는 대기
    driver.implicitly_wait(5)
    # 화면 스크롤
    for i in range(3) :
        driver.execute_script('window.scrollBy(0,1000)')
    time.sleep(5)
    # 후기 개수 확인 -> 적정선까지 로드
    for cnt in driver.find_elements_by_id('count') :
        print(cnt.text)
    print('-'*20)
    # 후기 채집 (작성자, 내용, 좋아요, 시간)
    for comment in driver.find_elements_by_tag_name('ytd-comment-thread-renderer') :
        print(comment.text)
        print('-'*20)

https://www.youtube.com/watch?v=roVed-6njCE

조회수 570회
--------------------
털이네온도니
1시간 전
무조건 플러스죠 .. 이번엔 더 심하게 플러스 사라는뜻인지 그냥 노트10은 해상도 충전속도 등 빠진게 너무많아요
2
답글
--------------------
공휴일궤
19분 전(수정됨)
전 실제로 만져보고..
그립감,무게,휴대성에서 압도적으로 좋은 노트10 주문했습니다.
휴대폰은 역시 휴대폰답게 휴대하기 좋아야 하니까요.
개인적으로 이전 노트시리즈는 너무 커서 안 샀었는데..
이번에는 작고 가벼운 것도 나와서 너무 좋네요~
자세히 보기
1
답글
--------------------
정보원
2시간 전
홍보해주겠다는데 돈받고 해주는사람 천지고마는 그걸 못찍게하노 어이그 덜된기라
1
답글
--------------------


In [62]:
# 한글명은 동일하게 세팅되니까, replace로 걷어내는 방법
# 정규식을 통해서 숫자만 남기고 모두 제거
tmp = ['조회수 39,529회', '댓글 362개']
tmp[0].replace('조회수 ', '').replace('회', ''), tmp[1].replace('댓글 ','').replace('개','')

('39,529', '362')

In [63]:
driver.close()
driver.quit()

In [ ]:
# sys.exit()